In [1]:
!pip install pyarrow

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
import pyarrow.parquet as pq
import s3fs
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Set up the S3 filesystem
s3 = s3fs.S3FileSystem(anon=False)

# List all the Parquet files in the S3 bucket/folder
s3_files = s3.glob('hugh-nguyen-belong-pedestrians-project-data/pedestrian_counts2/*.parquet')

# Open all the Parquet files
parquet_files = [pq.ParquetFile('s3://' + f, filesystem=s3) for f in s3_files]

# # Read the Parquet data into a pandas DataFrame
df = pd.concat([pf.read().to_pandas() for pf in parquet_files], ignore_index=True)

In [3]:
# Load reference data from S3
reference_files = s3.glob('hugh-nguyen-belong-pedestrians-project-data/sensor_reference_data/*.parquet')
reference_df = pd.concat([pq.ParquetFile('s3://' + f, filesystem=s3).read().to_pandas() for f in reference_files], ignore_index=True)

In [4]:
# Convert date_time column to datetime and extract date
df['date'] = pd.to_datetime(df['date_time']).dt.date
df.head(10)

,id,date_time,sensor_id,sensor_name,hourly_counts,date
0,2887628,2019-11-01T17:00:00,34,Flinders St-Spark La,300,2019-11-01
1,2887629,2019-11-01T17:00:00,39,Alfred Place,604,2019-11-01
2,2887630,2019-11-01T17:00:00,37,Lygon St (East),216,2019-11-01
3,2887631,2019-11-01T17:00:00,40,Lonsdale St-Spring St (West),627,2019-11-01
4,2887632,2019-11-01T17:00:00,36,Queen St (West),774,2019-11-01
5,2887633,2019-11-01T17:00:00,29,St Kilda Rd-Alexandra Gardens,644,2019-11-01
6,2887634,2019-11-01T17:00:00,42,Grattan St-Swanston St (West),453,2019-11-01
7,2887635,2019-11-01T17:00:00,43,Monash Rd-Swanston St (West),387,2019-11-01
8,2887636,2019-11-01T17:00:00,44,Tin Alley-Swanston St (West),27,2019-11-01
9,2887637,2019-11-01T17:00:00,35,Southbank,2691,2019-11-01


In [5]:
# # Convert location_id to int64 data type in reference data
# reference_df['location_id'] = reference_df['location_id'].astype('int64')
# df['sensor_id'] = df['sensor_id'].astype('int64')

In [6]:
top_sensors = df.groupby(['date', 'sensor_id'])['hourly_counts'].sum().reset_index(name='daily_count')
top_sensors.head(10)

,date,sensor_id,daily_count
0,2009-05-01,1,36869
1,2009-05-01,2,27587
2,2009-05-01,4,45185
3,2009-05-01,5,25590
4,2009-05-01,6,29015
5,2009-05-01,9,9243
6,2009-05-01,10,7877
7,2009-05-01,11,3864
8,2009-05-01,12,7462
9,2009-05-01,13,23350


In [7]:
# Prep the reference data for merging, and rename sensor_description to fit better with requirements
reference_df = reference_df.rename(columns={'location_id': 'sensor_id', 'sensor_description': 'location_name'})
reference_df.head(10)

,direction_1,direction_2,installation_date,latitude,location,sensor_id,location_type,longitude,note,location_name,sensor_name,status
0,East,West,2009-03-30,-37.813807,"[(lon, 144.96516718), (lat, -37.81380668)]",2,Outdoor,144.965167,None,Bourke Street Mall (South),Bou283_T,A
1,North,South,2009-03-23,-37.814880,"[(lon, 144.9660878), (lat, -37.81487988)]",4,Outdoor,144.966088,None,Town Hall (West),Swa123_T,A
2,North,South,2009-03-25,-37.819117,"[(lon, 144.96558255), (lat, -37.81911705)]",6,Outdoor,144.965583,Upgraded on 8/09/21,Flinders Street Station Underpass,FliS_T,A
3,North,South,2009-03-24,-37.822935,"[(lon, 144.9471751), (lat, -37.82293543)]",8,Outdoor,144.947175,None,Webb Bridge,WebBN_T,A
4,East,West,2009-04-23,-37.818765,"[(lon, 144.94710545), (lat, -37.81876474)]",10,Outdoor,144.947105,None,Victoria Point,BouHbr_T,A
5,East,West,2009-01-20,-37.815650,"[(lon, 144.93970694), (lat, -37.81564989)]",11,Outdoor,144.939707,None,Waterfront City,WatCit_T,A
6,North,South,2009-03-24,-37.820112,"[(lon, 144.96291897), (lat, -37.82011242)]",14,Outdoor,144.962919,Sensor relocated to sensor ID 25 on 2/10/2019,Sandridge Bridge,SanBri_T,A
7,East,West,2013-08-16,-37.806069,"[(lon, 144.95644692), (lat, -37.80606888)]",27,Outdoor,144.956447,None,QV Market-Peel St,Vic_T,A
8,North,South,2013-10-11,-37.819658,"[(lon, 144.96863453), (lat, -37.81965809)]",29,Outdoor,144.968635,sensor upgraded from laser to 3D on 19/12/2019,St Kilda Rd-Alexandra Gardens,AG_T,A
9,North,South,2013-10-10,-37.801697,"[(lon, 144.96658911), (lat, -37.80169681)]",31,Outdoor,144.966589,None,Lygon St (West),Lyg161_T,A


In [8]:
top_sensors = pd.merge(top_sensors, reference_df[['sensor_id', 'location_name']], on='sensor_id')
top_sensors.head(100)

,date,sensor_id,daily_count,location_name
0,2009-05-01,1,36869,Bourke Street Mall (North)
1,2009-05-02,1,30003,Bourke Street Mall (North)
2,2009-05-03,1,22327,Bourke Street Mall (North)
3,2009-05-04,1,26481,Bourke Street Mall (North)
4,2009-05-05,1,26671,Bourke Street Mall (North)
5,2009-05-06,1,27234,Bourke Street Mall (North)
6,2009-05-07,1,29064,Bourke Street Mall (North)
7,2009-05-08,1,37474,Bourke Street Mall (North)
8,2009-05-09,1,28397,Bourke Street Mall (North)
9,2009-05-10,1,19313,Bourke Street Mall (North)


In [9]:
# Compute top 10 sensor_ids by day ordered by day
top_sensors['rank'] = top_sensors.groupby('date')['daily_count'].rank(method='dense', ascending=False)
top_sensors = top_sensors.sort_values(['date', 'rank'], ascending=[False, True])
top_sensors = top_sensors.groupby('date').head(10)
top_sensors[['date','rank','sensor_id','location_name','daily_count']].head(1000)

,date,rank,sensor_id,location_name,daily_count
135164,2022-10-31,1.0,41,Flinders La-Swanston St (West),41206
13862,2022-10-31,2.0,4,Town Hall (West),35015
18654,2022-10-31,3.0,5,Princes Bridge,28340
4418,2022-10-31,4.0,1,Bourke Street Mall (North),25176
171353,2022-10-31,5.0,84,Elizabeth St - Flinders St (East) - New footpath,24036
61787,2022-10-31,6.0,3,Melbourne Central,22860
163673,2022-10-31,7.0,66,State Library - New,21295
133597,2022-10-31,8.0,35,Southbank,21090
144295,2022-10-31,9.0,47,Melbourne Central-Elizabeth St (East),19100
96529,2022-10-31,10.0,28,The Arts Centre,17537
